In [1]:
import pandas as pd
from os import listdir
from parsedatetime import parsedatetime
from datetime import datetime
from statistics import mode
# Working directory and date parser
wd = "Z:/PEDS/RI Biostatistics Core/Shared/Shared Projects/Laura/BDC/Projects/Erin Cobry/Prospective HCL and sleep study"
cal = parsedatetime.Calendar()

In [2]:
# Get file paths
files = listdir(wd + "./Data raw/CGM")
# Iterate through, clean, and calculate variables
results = {}
for f in files:
    # Read in
    path = wd + "/Data raw/CGM/" + f
    df = pd.read_csv(path)
    # Find columns
    time_col = [c for c in df.columns if "times" in c.lower()]
    gluc_col = [c for c in df.columns if "glucose value" in c.lower()]
    # Remove unnecessary columns
    clean = pd.concat([df[time_col],df[gluc_col]],axis = 1)
    clean.columns = ["time","glucose"]
    # Remove missing
    clean = clean[clean['time'].notnull()]
    # Format time
    clean["time"] = clean["time"].str.replace('T',' ')
    clean["time"] = [cal.parse(t)[0] for t in clean["time"]]
    clean["time"] = [datetime(*t[:6]) for t in clean["time"]]
    # Find sampling interval
    s = str(mode(clean["time"].diff()).total_seconds() / 60)
    # Re-index and expand to every 5 minutes
    clean.set_index("time",inplace = True)
    clean = clean[~clean.index.duplicated(keep='first')]
    clean = clean.resample(s + "T").ffill()
    clean.reset_index(inplace = True)
    # Write
    clean.to_csv(wd + "/Data clean/CGM/" + f,index = False)
    # Variables
    